In [2]:
import ee
import geemap
import pandas as pd
import os
import json
import geopandas as gpd
import geemap
import time

# Initialize
ee.Authenticate()
ee.Initialize(project='unicef-ccri')

ADM0

In [3]:
# Define admin level
admin_level = 'adm0'

# Set output folder
output_folder = f'p1_exposure_{admin_level}'


In [58]:

# Load child population and replace one image
org_childpop = ee.ImageCollection("projects/unicef-ccri/assets/WorldPop_Con_T_U18")
childpop = org_childpop.mosaic()
scale = org_childpop.first().projection().nominalScale()
totalpop = ee.Image("projects/unicef-ccri/assets/worldpop_1km")
totalpop_res = totalpop.projection().nominalScale()

reference_image = ee.Image("projects/unicef-ccri/assets/heatwave_frequency_2014_2023_avg")
target_scale = reference_image.projection().nominalScale()
target_crs = reference_image.projection().crs()

# Hazard list
hazards = [
    {"id": "projects/unicef-ccri/assets/river_flood_r100", "threshold": 0.01, "name": "river_flood_100yr_jrc_2024"},
    {"id": "projects/unicef-ccri/assets/coastal_flood_r100", "threshold": 0, "name": "coastal_flood_100yr_jrc_2024"},
    {"id": "projects/unicef-ccri/assets/storm_giri_rp100", "threshold": 17.5, "name": "tropical_storm_100yr_giri_2024"},
    {"id": "projects/unicef-ccri/assets/ASI_return_level_100yr", "threshold": 30, "name": "agricultural_drought_fao_1984-2023"},
    {"id": "projects/unicef-ccri/assets/drought_spei_copernicus_1940_2024", "threshold": -1.5, "name": "drought_spei_copernicus_1940-2024"},
    {"id": "projects/unicef-ccri/assets/drought_spi_copernicus_1940_2024", "threshold": -1.5, "name": "drought_spi_copernicus_1940-2024"},
    {"id": "projects/unicef-ccri/assets/heatwave_frequency_return_level_100yr", "threshold": 'Mean', "name": "heatwave_frequency_ecmwf_2014-2024"},
    {"id": "projects/unicef-ccri/assets/heatwave_duration_return_level_100yr", "threshold": 'Mean', "name": "heatwave_duration_ecmwf_2014-2024"},
    {"id": "projects/unicef-ccri/assets/heatwave_severity_return_level_100yr", "threshold": 'Mean', "name": "heatwave_severity_ecmwf_2014-2024"},
    {"id": "projects/unicef-ccri/assets/high_temp_degree_days_return_level_100yr", "threshold": 35, "name": "extreme_heat_ecmwf_2014-2024"},
    {"id": "projects/unicef-ccri/assets/FIRMS_FRP_90th_percentile", "threshold": 'Mean', "name": "fire_FRP_nasa_2001-2024"},
    {"id": "projects/unicef-ccri/assets/FIRMS_count_90th_percentile", "threshold": 'Mean', "name": "fire_frequency_nasa_2001-2023"},
    {"id": "projects/unicef-ccri/assets/sand_dust_storm_annual", "threshold": 0, "name": "sand_dust_storm_unccd_2024"},
    {"id": "projects/unicef-ccri/assets/pm25_p90_1998_2023", "threshold": 5, "name": "air_pollution_pm25_1998-2023"},
    {"id": "projects/unicef-ccri/assets/Pv_average_2013_2022", "threshold": 0.001, "name": "vectorborne_malariapv_2012-2022"},
    {"id": "projects/unicef-ccri/assets/Pf_average_2013_2022", "threshold": 0.001, "name": "vectorborne_malariapf_2012-2022"}
]

# Load country boundaries
admin_fc_path = f'projects/unicef-ccri/assets/georepo_{admin_level}'
country_fc = ee.FeatureCollection(admin_fc_path)
simple_fc = ee.FeatureCollection(f'projects/unicef-ccri/assets/{admin_level}_simple')
area_fc = ee.FeatureCollection(f'projects/unicef-ccri/assets/{admin_level}_wfp')
adm_ids = country_fc.aggregate_array('WFP_ID').distinct().getInfo()
global_geom = simple_fc.geometry()

# Group features by 'WFP_ID' and merge geometries
merged_fc = country_fc.distinct(['WFP_ID']).map(lambda f: f.setGeometry(
      country_fc
        .filter(ee.Filter.eq('WFP_ID', f.get('WFP_ID')))
        .geometry()
        .dissolve()
  ))


# Update thresholds if needed
for hazard in hazards:
    if hazard['threshold'] == 'Mean':
        layer = (
            ee.ImageCollection(hazard['id']).mosaic()
            if hazard['name'] in ["river_flood_100yr_jrc_2024", "coastal_flood_100yr_jrc_2024", "tropical_storm_100yr_giri_2024"]
            else ee.Image(hazard['id'])
        )
        th = layer.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=global_geom,
            scale=target_scale,
            bestEffort=True,
            maxPixels=1e13
        ).values().get(0)
        if th is not None:
            hazard['threshold'] = ee.Number(th).getInfo()
        else:
            print(f"⚠️ Skipping hazard {hazard['name']} — global mean threshold is null.")
            hazard['skip'] = True


th_shape_area = area_fc.filter(ee.Filter.eq('iso3', 'ITA')).first().getNumber('Shape_Area')

# Loop through admin regions
for adm_id in adm_ids:
    print(adm_id)
    if adm_id is None:
        continue

    # Filter feature by adm_id
    region = merged_fc.filter(ee.Filter.eq('WFP_ID', adm_id)).first()
    area_region = area_fc.filter(ee.Filter.eq('adm0_id', int(adm_id))).first()

    print(f"Submitting task for {admin_level}_id: {adm_id}...")

    shape_area = area_region.getNumber('Shape_Area')

    # Choose simplification level based on area
    simplification_tolerance = ee.Algorithms.If(
        shape_area.gt(th_shape_area),
        10000,
        100
    )

    simplified_geom = region.geometry().simplify(simplification_tolerance)
    region_geom = ee.Geometry(simplified_geom)

    iso3 = region.get('ISO3')
    wfp_id = region.get('WFP_ID')
    region_name = region.get(f'name')
    region_type = region.get(f'type')

    childpop_sum = childpop.reduceRegion(
        ee.Reducer.sum(), region_geom, scale=scale, crs='EPSG:4326', maxPixels=1e13).get('b1')
    totalpop_sum = totalpop.reduceRegion(
        ee.Reducer.sum(), region_geom, scale=totalpop_res, crs='EPSG:4326', maxPixels=1e13).get('b1')

    features = []

    for hazard in hazards:
        if hazard.get('skip'):
            continue
        name = hazard['name']
        TH = hazard['threshold']

        layer = (
            ee.ImageCollection(hazard['id']).mosaic()
            if hazard['name'] in ["river_flood_100yr_jrc_2024", "coastal_flood_100yr_jrc_2024", "tropical_storm_100yr_giri_2024"]
            else ee.Image(hazard['id'])
        )

        if name == "agricultural_drought_fao_1984-2023":
            layer = layer.updateMask(layer.lte(100))
            exposed = childpop.updateMask(layer.gt(ee.Image.constant(TH)))
        elif name in ["drought_spei_copernicus_1940-2024", "drought_spi_copernicus_1940-2024"]:
            exposed = childpop.updateMask(layer.lt(ee.Image.constant(TH)))
        else:
            exposed = childpop.updateMask(layer.gt(ee.Image.constant(TH)))


        if name == "coastal_flood_100yr_jrc_2024":
              region_geom = region_geom.buffer(5000) #buffer 5km for coastal flood
        else:
              region_geom = ee.Geometry(simplified_geom)


        exposed_sum = exposed.reduceRegion(
            ee.Reducer.sum(), region_geom, scale=scale, bestEffort=True, crs='EPSG:4326', maxPixels=1e13).get('b1')

        feature = ee.Feature(None, {

            'iso3': iso3,
            'wfp_id': wfp_id,
            f'{admin_level}_name': region_name,
            'type' : region_type,
            'hazard': name,
            'child_population_exposed': exposed_sum,
            'child_population_total': childpop_sum,
            'population_total': totalpop_sum
        })
        features.append(feature)

    task = ee.batch.Export.table.toDrive(
        collection=ee.FeatureCollection(features),
        description=f"{adm_id}_hazards_{admin_level}",
        folder=output_folder,
        fileFormat='CSV',
        selectors=[
            'iso3', 'wfp_id', f'{admin_level}_name', 'type', 'hazard',
            'child_population_exposed', 'child_population_total', 'population_total'
        ]
    )
    task.start()


print("✅ All tasks submitted.")


9
Submitting task for adm0_id: 9...
268
Submitting task for adm0_id: 268...
87
Submitting task for adm0_id: 87...
55
Submitting task for adm0_id: 55...
178
Submitting task for adm0_id: 178...
30
Submitting task for adm0_id: 30...
95
Submitting task for adm0_id: 95...
39
Submitting task for adm0_id: 39...
48
Submitting task for adm0_id: 48...
168
Submitting task for adm0_id: 168...
197
Submitting task for adm0_id: 197...
207
Submitting task for adm0_id: 207...
1017
Submitting task for adm0_id: 1017...
1019
Submitting task for adm0_id: 1019...
1018
Submitting task for adm0_id: 1018...
244
Submitting task for adm0_id: 244...
251
Submitting task for adm0_id: 251...
5
Submitting task for adm0_id: 5...
101
Submitting task for adm0_id: 101...
258
Submitting task for adm0_id: 258...
98
Submitting task for adm0_id: 98...
283
Submitting task for adm0_id: 283...
15
Submitting task for adm0_id: 15...
218
Submitting task for adm0_id: 218...
136
Submitting task for adm0_id: 136...
282
Submitting tas

ADM1

In [ ]:

# Load child population and replace one image
org_childpop = ee.ImageCollection("projects/unicef-ccri/assets/childpop_constrained")
image_to_replace_id = 'tha_T_Under_18_2024_CN_100m_R2024A_v1'
new_image = ee.Image(f'projects/unicef-ccri/assets/{image_to_replace_id}').set('system:index', image_to_replace_id)
filtered_collection = org_childpop.filter(ee.Filter.neq('system:index', image_to_replace_id))
childpop = filtered_collection.merge(ee.ImageCollection([new_image])).mosaic()
scale = filtered_collection.first().projection().nominalScale()
totalpop = ee.Image("projects/unicef-ccri/assets/worldpop_1km")
totalpop_res = totalpop.projection().nominalScale()

reference_image = ee.Image("projects/unicef-ccri/assets/heatwave_frequency_2014_2023_avg")
target_scale = reference_image.projection().nominalScale()
target_crs = reference_image.projection().crs()

# Hazard list
hazards = [
    {"id": "projects/unicef-ccri/assets/river_flood_r100", "threshold": 0.01, "name": "river_flood_100yr_jrc_2024"},
    {"id": "projects/unicef-ccri/assets/coastal_flood_r100", "threshold": 0, "name": "coastal_flood_100yr_jrc_2024"},
    {"id": "projects/unicef-ccri/assets/storm_giri_rp100", "threshold": 17.5, "name": "tropical_storm_100yr_giri_2024"},
    {"id": "projects/unicef-ccri/assets/ASI_return_level_100yr", "threshold": 30, "name": "agricultural_drought_fao_1984-2023"},
    {"id": "projects/unicef-ccri/assets/drought_spei_copernicus_1940_2024", "threshold": -1.5, "name": "drought_spei_copernicus_1940-2024"},
    {"id": "projects/unicef-ccri/assets/drought_spi_copernicus_1940_2024", "threshold": -1.5, "name": "drought_spi_copernicus_1940-2024"},
    {"id": "projects/unicef-ccri/assets/heatwave_frequency_return_level_100yr", "threshold": 'Mean', "name": "heatwave_frequency_ecmwf_2014-2024"},
    {"id": "projects/unicef-ccri/assets/heatwave_duration_return_level_100yr", "threshold": 'Mean', "name": "heatwave_duration_ecmwf_2014-2024"},
    {"id": "projects/unicef-ccri/assets/heatwave_severity_return_level_100yr", "threshold": 'Mean', "name": "heatwave_severity_ecmwf_2014-2024"},
    {"id": "projects/unicef-ccri/assets/high_temp_degree_days_return_level_100yr", "threshold": 35, "name": "extreme_heat_ecmwf_2014-2024"},
    {"id": "projects/unicef-ccri/assets/FIRMS_FRP_90th_percentile", "threshold": 'Mean', "name": "fire_FRP_nasa_2001-2024"},
    {"id": "projects/unicef-ccri/assets/FIRMS_count_90th_percentile", "threshold": 'Mean', "name": "fire_frequency_nasa_2001-2023"},
    {"id": "projects/unicef-ccri/assets/sand_dust_storm_annual", "threshold": 0, "name": "sand_dust_storm_unccd_2024"},
    {"id": "projects/unicef-ccri/assets/pm25_p90_1998_2023", "threshold": 5, "name": "air_pollution_pm25_1998-2023"},
    {"id": "projects/unicef-ccri/assets/Pv_average_2013_2022", "threshold": 0.001, "name": "vectorborne_malariapv_2012-2022"},
    {"id": "projects/unicef-ccri/assets/Pf_average_2013_2022", "threshold": 0.001, "name": "vectorborne_malariapf_2012-2022"}
]

# Load country boundaries

GAUL_fc_path = f'projects/sat-io/open-datasets/FAO/GAUL/GAUL_2024_L{admin_level}'
WFP_fc_path = f'projects/unicef-ccri/assets/adm0_wfp'

# Load FeatureCollections
gaul_fc = ee.FeatureCollection(GAUL_fc_path)
wfp_fc = ee.FeatureCollection(WFP_fc_path)

# Define spatial join
spatial_filter = ee.Filter.intersects(leftField='.geo', rightField='.geo')
save_all_join = ee.Join.saveAll(matchesKey='matched_wfp')

# Apply the join: attach intersecting WFP features to each GAUL feature
joined_fc = save_all_join.apply(primary=gaul_fc, secondary=wfp_fc, condition=spatial_filter)


adm_ids = joined_fc.aggregate_array(f'gaul{admin_level}_code').distinct().getInfo()

simple_fc = ee.FeatureCollection(f'projects/unicef-ccri/assets/adm0_simple')
global_geom = simple_fc.geometry()

# Update thresholds if needed
for hazard in hazards:
    if hazard['threshold'] == 'Mean':
        layer = (
            ee.ImageCollection(hazard['id']).mosaic()
            if hazard['name'] in ["river_flood_100yr_jrc_2024", "coastal_flood_100yr_jrc_2024", "tropical_storm_100yr_giri_2024"]
            else ee.Image(hazard['id'])
        )
        th = layer.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=global_geom,
            scale=target_scale,
            bestEffort=True,
            maxPixels=1e13
        ).values().get(0)
        if th is not None:
            hazard['threshold'] = ee.Number(th).getInfo()
        else:
            print(f"⚠️ Skipping hazard {hazard['name']} — global mean threshold is null.")
            hazard['skip'] = True


th_shape_area = wfp_fc.filter(ee.Filter.eq(f'adm0_id', 122)).first().geometry().area()
#thresh = th_shape_area.getInfo()
# Function to compute intersection area for each WFP match
def score_overlap(wfp_feature):
    wfp_geom = ee.Feature(wfp_feature).geometry()
    intersection = region.geometry().intersection(wfp_geom, ee.ErrorMargin(1))
    return ee.Feature(wfp_feature).set('overlap_area', intersection.area())


# Loop through admin regions
for adm_id in adm_ids:
#for i in range(1):  # Loop runs only once
    # adm_id = adm_ids[i]
    print(f"Processing adm_id: {adm_id}...")
    if adm_id is None:
        continue

    # Filter feature by adm_id
    region = joined_fc.filter(ee.Filter.eq(f'gaul{admin_level}_code', adm_id)).first()

    try:
        #region_info = region.getInfo()
        matched_wfp = ee.List(region.get('matched_wfp'))
        # Map over matched WFP features
        scored = ee.FeatureCollection(matched_wfp.map(score_overlap))
        # Get the one with the largest overlap area
        best_matched = scored.sort('overlap_area', False).first()

        #if matched_wfp.size().getInfo() > 0:
        stscod = best_matched.get('stscod')
        wfp_iso3 = best_matched.get('iso3')
        wfp_name = best_matched.get('adm0_name')
    except Exception as e:
        print(f"⚠️ Could not retrieve region for adm_id {adm_id}. Error: {e}")
        stscod = None
        wfp_iso3 = None
        wfp_name = None

    # Skip if stscod is not 'State'
    if stscod in  ['Territory','Non-Self Governing Territory']:
        print(f"Skipping adm_id {adm_id} due to stscod = {stscod}")
        continue

    print(f"Submitting task for {admin_level}_id: {adm_id}...")

    shape_area = region.geometry().area()

    region_geom = ee.Geometry(
        ee.Algorithms.If(
            shape_area.gt(th_shape_area),
            region.geometry().simplify(10000),
            region.geometry()
        )
    )

    #region_geom = region.geometry().simplify(10000)

    iso3 = region.get('iso3_code')
    region_name = region.get('gaul0_name')

    childpop_sum = childpop.reduceRegion(
        ee.Reducer.sum(), region_geom, scale=scale, crs='EPSG:4326', maxPixels=1e13).get('b1')
    totalpop_sum = totalpop.reduceRegion(
        ee.Reducer.sum(), region_geom, scale=totalpop_res, crs='EPSG:4326', maxPixels=1e13).get('b1')

    features = []

    for hazard in hazards:
        if hazard.get('skip'):
            continue
        name = hazard['name']
        TH = hazard['threshold']

        layer = (
            ee.ImageCollection(hazard['id']).mosaic()
            if hazard['name'] in ["river_flood_100yr_jrc_2024", "coastal_flood_100yr_jrc_2024", "tropical_storm_100yr_giri_2024"]
            else ee.Image(hazard['id'])
        )
        ##if the country size is too small resample the hazard layer to the resoulution of population data to prevent zero count
        # if shape_area.lt(th_shape_area):
        #     layer = layer.clip(region_geom).resample('nearest').reproject(crs='EPSG:4326', scale=scale)

        if name == "agricultural_drought_fao_1984-2023":
            layer = layer.updateMask(layer.lte(100))
            exposed = childpop.updateMask(layer.gt(ee.Image.constant(TH)))
        elif name in ["drought_spei_copernicus_1940-2024", "drought_spi_copernicus_1940-2024"]:
            exposed = childpop.updateMask(layer.lt(ee.Image.constant(TH)))
        else:
            exposed = childpop.updateMask(layer.gt(ee.Image.constant(TH)))

        exposed_sum = exposed.reduceRegion(
            ee.Reducer.sum(), region_geom, scale=scale, bestEffort=True, crs='EPSG:4326', maxPixels=1e13).get('b1')

        feature = ee.Feature(None, {
            'gaul0_iso3': iso3,
            'gaul0_name': region_name,
            'gaul1_code': adm_id,
            'wfp_iso3': wfp_iso3,
            'wfp_name': wfp_name,
            'hazard': name,
            'child_population_exposed': exposed_sum,
            'child_population_total': childpop_sum,
            'population_total': totalpop_sum
        })
        features.append(feature)

    task = ee.batch.Export.table.toDrive(
        collection=ee.FeatureCollection(features),
        description=f"adm{adm_id}_hazards_adm{admin_level}",
        folder=output_folder,
        fileFormat='CSV',
        selectors=[
            'gaul0_iso3', 'gaul0_name', 'gaul1_code', 'wfp_iso3', 'wfp_name', 'hazard',
            'child_population_exposed', 'child_population_total', 'population_total'
        ]
    )
    task.start()


print("✅ All tasks submitted.")


Streaming output truncated to the last 5000 lines.
Submitting task for 1_id: 1611...
Processing adm_id: 1612...
Submitting task for 1_id: 1612...
Processing adm_id: 1613...
Submitting task for 1_id: 1613...
Processing adm_id: 1614...
Submitting task for 1_id: 1614...
Processing adm_id: 1615...
Submitting task for 1_id: 1615...
Processing adm_id: 1616...
Submitting task for 1_id: 1616...
Processing adm_id: 1617...
Submitting task for 1_id: 1617...
Processing adm_id: 1618...
Submitting task for 1_id: 1618...
Processing adm_id: 1619...
Submitting task for 1_id: 1619...
Processing adm_id: 1620...
Submitting task for 1_id: 1620...
Processing adm_id: 1621...
Submitting task for 1_id: 1621...
Processing adm_id: 1622...
Submitting task for 1_id: 1622...
Processing adm_id: 1623...
Submitting task for 1_id: 1623...
Processing adm_id: 1624...
Submitting task for 1_id: 1624...
Processing adm_id: 1625...
Submitting task for 1_id: 1625...
Processing adm_id: 1626...
Submitting task for 1_id: 1626...


Combining into adm0

In [67]:
import os
import glob
import pandas as pd

# Define the folder and get all CSV file paths
hazard_folder = os.path.join('/content/drive/MyDrive', output_folder)
hazard_files = glob.glob(f'{hazard_folder}/*.csv')

# Read and concatenate all CSVs into one DataFrame
merged_df = pd.concat([pd.read_csv(f) for f in hazard_files], ignore_index=True)

# Output directory
output_dir = '/content/drive/MyDrive/p1_exposure'
os.makedirs(output_dir, exist_ok=True)

# Loop through each unique hazard and export a CSV
for hazard_name in merged_df['hazard'].unique():
    # Filter the data
    df_subset = merged_df[merged_df['hazard'] == hazard_name]

    # Define full path
    output_path = os.path.join(output_dir, f"{hazard_name}_exposure_adm0.csv")

    # Save the CSV
    df_subset.to_csv(output_path, index=False)

    print(f"✅ Saved: {output_path}")


✅ Saved: /content/drive/MyDrive/p1_exposure/river_flood_100yr_jrc_2024_exposure_adm0.csv
✅ Saved: /content/drive/MyDrive/p1_exposure/coastal_flood_100yr_jrc_2024_exposure_adm0.csv
✅ Saved: /content/drive/MyDrive/p1_exposure/tropical_storm_100yr_giri_2024_exposure_adm0.csv
✅ Saved: /content/drive/MyDrive/p1_exposure/agricultural_drought_fao_1984-2023_exposure_adm0.csv
✅ Saved: /content/drive/MyDrive/p1_exposure/drought_spei_copernicus_1940-2024_exposure_adm0.csv
✅ Saved: /content/drive/MyDrive/p1_exposure/drought_spi_copernicus_1940-2024_exposure_adm0.csv
✅ Saved: /content/drive/MyDrive/p1_exposure/heatwave_frequency_ecmwf_2014-2024_exposure_adm0.csv
✅ Saved: /content/drive/MyDrive/p1_exposure/heatwave_duration_ecmwf_2014-2024_exposure_adm0.csv
✅ Saved: /content/drive/MyDrive/p1_exposure/heatwave_severity_ecmwf_2014-2024_exposure_adm0.csv
✅ Saved: /content/drive/MyDrive/p1_exposure/extreme_heat_ecmwf_2014-2024_exposure_adm0.csv
✅ Saved: /content/drive/MyDrive/p1_exposure/fire_FRP_nasa_2

In [ ]:

# Load child population and replace one image
org_childpop = ee.ImageCollection("projects/unicef-ccri/assets/childpop_constrained")
image_to_replace_id = 'tha_T_Under_18_2024_CN_100m_R2024A_v1'
new_image = ee.Image(f'projects/unicef-ccri/assets/{image_to_replace_id}').set('system:index', image_to_replace_id)
filtered_collection = org_childpop.filter(ee.Filter.neq('system:index', image_to_replace_id))
childpop = filtered_collection.merge(ee.ImageCollection([new_image])).mosaic()
scale = filtered_collection.first().projection().nominalScale()
totalpop = ee.Image("projects/unicef-ccri/assets/worldpop_1km")
totalpop_res = totalpop.projection().nominalScale()

reference_image = ee.Image("projects/unicef-ccri/assets/heatwave_frequency_2014_2023_avg")
target_scale = reference_image.projection().nominalScale()
target_crs = reference_image.projection().crs()

# Hazard list
hazards = [
    {"id": "projects/unicef-ccri/assets/river_flood_r100", "threshold": 0.01, "name": "river_flood_100yr_jrc_2024"},
    {"id": "projects/unicef-ccri/assets/coastal_flood_r100", "threshold": 0, "name": "coastal_flood_100yr_jrc_2024"},
    {"id": "projects/unicef-ccri/assets/storm_giri_rp100", "threshold": 17.5, "name": "tropical_storm_100yr_giri_2024"},
    {"id": "projects/unicef-ccri/assets/ASI_return_level_100yr", "threshold": 30, "name": "agricultural_drought_fao_1984-2023"},
    {"id": "projects/unicef-ccri/assets/drought_spei_copernicus_1940_2024", "threshold": -1.5, "name": "drought_spei_copernicus_1940-2024"},
    {"id": "projects/unicef-ccri/assets/drought_spi_copernicus_1940_2024", "threshold": -1.5, "name": "drought_spi_copernicus_1940-2024"},
    {"id": "projects/unicef-ccri/assets/heatwave_frequency_return_level_100yr", "threshold": 'Mean', "name": "heatwave_frequency_ecmwf_2014-2024"},
    {"id": "projects/unicef-ccri/assets/heatwave_duration_return_level_100yr", "threshold": 'Mean', "name": "heatwave_duration_ecmwf_2014-2024"},
    {"id": "projects/unicef-ccri/assets/heatwave_severity_return_level_100yr", "threshold": 'Mean', "name": "heatwave_severity_ecmwf_2014-2024"},
    {"id": "projects/unicef-ccri/assets/high_temp_degree_days_return_level_100yr", "threshold": 35, "name": "extreme_heat_ecmwf_2014-2024"},
    {"id": "projects/unicef-ccri/assets/FIRMS_FRP_90th_percentile", "threshold": 'Mean', "name": "fire_FRP_nasa_2001-2024"},
    {"id": "projects/unicef-ccri/assets/FIRMS_count_90th_percentile", "threshold": 'Mean', "name": "fire_frequency_nasa_2001-2023"},
    {"id": "projects/unicef-ccri/assets/sand_dust_storm_annual", "threshold": 0, "name": "sand_dust_storm_unccd_2024"},
    {"id": "projects/unicef-ccri/assets/pm25_p90_1998_2023", "threshold": 5, "name": "air_pollution_pm25_1998-2023"},
    {"id": "projects/unicef-ccri/assets/Pv_average_2013_2022", "threshold": 0.001, "name": "vectorborne_malariapv_2012-2022"},
    {"id": "projects/unicef-ccri/assets/Pf_average_2013_2022", "threshold": 0.001, "name": "vectorborne_malariapf_2012-2022"}
]

# Load country boundaries
admin_fc_path = f'projects/unicef-ccri/assets/gaul2024_{admin_level}'
country_fc = ee.FeatureCollection(admin_fc_path)
simple_fc = ee.FeatureCollection(f'projects/unicef-ccri/assets/{admin_level}_simple')
adm_ids = country_fc.aggregate_array('WFP_ID').distinct().getInfo()
global_geom = simple_fc.geometry()

# Group features by 'WFP_ID' and merge geometries
merged_fc = country_fc.distinct(['WFP_ID']).map(lambda f: f.setGeometry(
      country_fc
        .filter(ee.Filter.eq('WFP_ID', f.get('WFP_ID')))
        .geometry()
        .dissolve()
  ))



# Update thresholds if needed
for hazard in hazards:
    if hazard['threshold'] == 'Mean':
        layer = (
            ee.ImageCollection(hazard['id']).mosaic()
            if hazard['name'] in ["river_flood_100yr_jrc_2024", "coastal_flood_100yr_jrc_2024", "tropical_storm_100yr_giri_2024"]
            else ee.Image(hazard['id'])
        )
        th = layer.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=global_geom,
            scale=target_scale,
            bestEffort=True,
            maxPixels=1e13
        ).values().get(0)
        if th is not None:
            hazard['threshold'] = ee.Number(th).getInfo()
        else:
            print(f"⚠️ Skipping hazard {hazard['name']} — global mean threshold is null.")
            hazard['skip'] = True


th_shape_area = merged_fc.filter(ee.Filter.eq('iso3_code', 'ITA')).first().getNumber('Shape_Area')

# Loop through admin regions
for adm_id in adm_ids:
    print(adm_id)
    if adm_id is None:
        continue

    # Filter feature by adm_id
    region = merged_fc.filter(ee.Filter.eq('WFP_ID', adm_id)).first()

    # Get stscod
    stscod = region.get('stscod').getInfo()

    print(f"Submitting task for {admin_level}_id: {adm_id}...")

    shape_area = region.getNumber('Shape_Area')

    # Choose simplification level based on area
    simplification_tolerance = ee.Algorithms.If(
        shape_area.gt(th_shape_area),
        10000,
        100
    )

    simplified_geom = region.geometry().simplify(simplification_tolerance)
    region_geom = ee.Geometry(simplified_geom)

    iso3_gaul = region.get('iso3_code')
    iso3_wfp = region.get('ISO3')
    wfp_id = region.get('WFP_ID')
    region_name = region.get(f'name')
    region_type = region.get(f'type')

    childpop_sum = childpop.reduceRegion(
        ee.Reducer.sum(), region_geom, scale=scale, crs='EPSG:4326', maxPixels=1e13).get('b1')
    totalpop_sum = totalpop.reduceRegion(
        ee.Reducer.sum(), region_geom, scale=totalpop_res, crs='EPSG:4326', maxPixels=1e13).get('b1')

    features = []

    for hazard in hazards:
        if hazard.get('skip'):
            continue
        name = hazard['name']
        TH = hazard['threshold']

        layer = (
            ee.ImageCollection(hazard['id']).mosaic()
            if hazard['name'] in ["river_flood_100yr_jrc_2024", "coastal_flood_100yr_jrc_2024", "tropical_storm_100yr_giri_2024"]
            else ee.Image(hazard['id'])
        )

        if name == "agricultural_drought_fao_1984-2023":
            layer = layer.updateMask(layer.lte(100))
            exposed = childpop.updateMask(layer.gt(ee.Image.constant(TH)))
        elif name in ["drought_spei_copernicus_1940-2024", "drought_spi_copernicus_1940-2024"]:
            exposed = childpop.updateMask(layer.lt(ee.Image.constant(TH)))
        else:
            exposed = childpop.updateMask(layer.gt(ee.Image.constant(TH)))


        if name == "coastal_flood_100yr_jrc_2024":
              region_geom = region_geom.buffer(5000) #buffer 5km for coastal flood
        else:
              region_geom = ee.Geometry(simplified_geom)


        exposed_sum = exposed.reduceRegion(
            ee.Reducer.sum(), region_geom, scale=scale, bestEffort=True, crs='EPSG:4326', maxPixels=1e13).get('b1')

        feature = ee.Feature(None, {

            'iso3': iso3_gaul,
            'iso3_wfp': iso3_wfp,
            'wfp_id': wfp_id,
            f'{admin_level}_name': region_name,
            'type' : region_type,
            'hazard': name,
            'child_population_exposed': exposed_sum,
            'child_population_total': childpop_sum,
            'population_total': totalpop_sum
        })
        features.append(feature)

    task = ee.batch.Export.table.toDrive(
        collection=ee.FeatureCollection(features),
        description=f"{adm_id}_hazards_{admin_level}",
        folder=output_folder,
        fileFormat='CSV',
        selectors=[
            'iso3', 'iso3_wfp', 'wfp_id', f'{admin_level}_name', 'type', 'hazard',
            'child_population_exposed', 'child_population_total', 'population_total'
        ]
    )
    task.start()


print("✅ All tasks submitted.")
